In [1]:
#############################################
#     Challenge Part 2 Locate Hotels        #
#############################################

In [2]:
# enable plotting in the Jupyter Notebook
%matplotlib inline

# Import the dependencies.
import pandas as pd
import gmaps
import gmaps.datasets
import requests

# Import the google API key.
from config import g_key

In [3]:
#############################################
#  Read City Weather Data Into a DataFrame  #
#############################################

In [4]:
# Read city weather information
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
0,0,Punta Arenas,CL,2020-05-10 00:40:56,-53.15,-70.92,44.60,93,90,3.36,overcast clouds,0,0
1,1,Hilo,US,2020-05-10 00:40:38,19.73,-155.09,80.60,57,40,8.05,scattered clouds,0,0
2,2,Norman Wells,CA,2020-05-10 00:45:02,65.28,-126.83,48.20,45,5,18.34,clear sky,0,0
3,3,Grand Gaube,MU,2020-05-10 00:45:02,-20.01,57.66,70.00,82,36,5.01,scattered clouds,0,0
4,4,Butaritari,KI,2020-05-10 00:40:44,3.07,172.79,83.68,78,31,9.22,scattered clouds,0,0


In [5]:
#############################################
#    Select Destinations Based on Weather   #
#############################################

In [6]:
# Ask the customer the desired weather condition at the vacation destination.
destination_min_temp  = float(input("What is the minimum temperature at your vacation destination? "))
destination_max_temp  = float(input("What is the maximum temperature at your vacation destination? "))
destination_want_rain = input("Do you want it to be raining at your vacation destination? (yes/no) ")
destination_want_snow = input("Do you want it to be snowing at your vacation destination? (yes/no) ")

if destination_want_rain is None:
    destination_want_rain = "no"
    
if destination_want_snow is None:
    destination_want_snow = "no"

print()
print(f"Searching for destinations where the current temperature is between {destination_min_temp} and {destination_max_temp}")   
print(f"Show destinations where it is currently raining [{destination_want_rain}] or currently snowing [{destination_want_snow}]")

What is the minimum temperature at your vacation destination? 70
What is the maximum temperature at your vacation destination? 100
Do you want it to be raining at your vacation destination? (yes/no) no
Do you want it to be snowing at your vacation destination? (yes/no) no

Searching for destinations where the current temperature is between 70.0 and 100.0
Show destinations where it is currently raining [no] or currently snowing [no]


In [7]:
# Filter the dataset to find the cities that fit the criteria.
if ( destination_want_rain.lower() != "yes" and destination_want_snow.lower() != "yes"):
    
    print("Looking for destinations where it IS NOT raining and IS NOT snowing")
    
    preferred_cities_df = city_data_df.loc[
                                    (city_data_df["Max Temp"] <= destination_max_temp) & 
                                    (city_data_df["Max Temp"] >= destination_min_temp) & 
                                    (city_data_df["Rain Inches (last 3 hrs)"] == 0) & 
                                    (city_data_df["Snow Inches (last 3 hrs)"] == 0), :]

elif destination_want_rain.lower() == "yes":
    
    print("Looking for destinations where it IS raining")
    
    preferred_cities_df = city_data_df.loc[
                                    (city_data_df["Max Temp"] <= destination_max_temp) & 
                                    (city_data_df["Max Temp"] >= destination_min_temp) & 
                                    (city_data_df["Rain Inches (last 3 hrs)"] > 0) &
                                    (city_data_df["Snow Inches (last 3 hrs)"] == 0), :]
    
elif len(destination_want_snow) > 0 and destination_want_snow.lower() == "yes":
    
    print("Looking for destinations where it IS snowing")
    
    preferred_cities_df = city_data_df.loc[
                                    (city_data_df["Max Temp"] <= destination_max_temp) & 
                                    (city_data_df["Max Temp"] >= destination_min_temp) & 
                                    (city_data_df["Rain Inches (last 3 hrs)"] == 0) &
                                    (city_data_df["Snow Inches (last 3 hrs)"] > 0), :]


Looking for destinations where it IS NOT raining and IS NOT snowing


In [8]:
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
1,1,Hilo,US,2020-05-10 00:40:38,19.73,-155.09,80.60,57,40,8.05,scattered clouds,0,0
3,3,Grand Gaube,MU,2020-05-10 00:45:02,-20.01,57.66,70.00,82,36,5.01,scattered clouds,0,0
4,4,Butaritari,KI,2020-05-10 00:40:44,3.07,172.79,83.68,78,31,9.22,scattered clouds,0,0
5,5,Wuzhou,CN,2020-05-10 00:45:04,23.48,111.32,85.87,75,38,3.71,moderate rain,0,0
6,6,Vaini,TO,2020-05-10 00:41:11,-21.20,-175.20,80.60,94,75,10.29,thunderstorm,0,0
12,12,Matara,LK,2020-05-10 00:45:05,5.95,80.54,80.29,88,96,3.58,overcast clouds,0,0
13,13,Avarua,CK,2020-05-10 00:41:01,-21.21,-159.78,82.40,74,0,14.99,clear sky,0,0
18,18,Rikitea,PF,2020-05-10 00:40:43,-23.12,-134.97,75.92,69,6,3.83,clear sky,0,0
20,20,Hithadhoo,MV,2020-05-10 00:45:06,-0.60,73.08,84.63,76,99,11.30,light rain,0,0
22,22,Lishu,CN,2020-05-10 00:45:06,43.30,124.33,70.02,45,0,14.97,clear sky,0,0


In [9]:
##########################################################
# Add Hotel Name Column to Perferred Cities Data Frame   #
##########################################################

In [10]:
# Create DataFrame to store hotel names along with city, country, current weather discription, and coordinates.
preferred_cities_w_hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Add hotel name field to the data frame.
preferred_cities_w_hotel_df["Hotel Name"] = ""

preferred_cities_w_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hilo,US,80.60,scattered clouds,19.73,-155.09,
3,Grand Gaube,MU,70.00,scattered clouds,-20.01,57.66,
4,Butaritari,KI,83.68,scattered clouds,3.07,172.79,
5,Wuzhou,CN,85.87,moderate rain,23.48,111.32,
6,Vaini,TO,80.60,thunderstorm,-21.20,-175.20,
12,Matara,LK,80.29,overcast clouds,5.95,80.54,
13,Avarua,CK,82.40,clear sky,-21.21,-159.78,
18,Rikitea,PF,75.92,clear sky,-23.12,-134.97,
20,Hithadhoo,MV,84.63,light rain,-0.60,73.08,
22,Lishu,CN,70.02,clear sky,43.30,124.33,


In [11]:
##############################################
# For Selected Destination Get Nearby Hotels #
##############################################

In [12]:
# Google nearby search URL endpoint.
url_google_nearbysearch = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

hotels_found_count = 0
city_name_padding = preferred_cities_w_hotel_df['City'].str.len().max()

# Create parameter object for nearby search.
params = {
    "radius": 5000, # in meeters
    "types": "lodging", # Only looking for hotels
    "key": g_key,
    "location": "Lat:Lng"
         }



# Iterate through the preferred cities data frame
# and add hotel information.
for index, row in preferred_cities_w_hotel_df.iterrows():
    
    # Get current data frame city's latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Update the location key in the request parameters.
    params["location"] = f"{lat},{lng}"
   
    # Request nearby search for hotels.
    hotels_nearby_results = requests.get(url_google_nearbysearch, params=params).json()
 
    # Get name of the first hotel in the the nearby search result set.
    try:        
        hotel_name = hotels_nearby_results["results"][0]["name"]
        print("City: " + row['City'].ljust(city_name_padding) + "  Hotel: " + hotel_name)
        preferred_cities_w_hotel_df.loc[index, "Hotel Name"] = hotel_name
        hotels_found_count +=1
    except:
        print("City: " + row['City'].ljust(city_name_padding) + "  Hotel: *** NO HOTEL FOUND *** ... Dropping row.")
        preferred_cities_w_hotel_df.drop([index])

City: Hilo                 Hotel: Hilo Hawaiian Hotel
City: Grand Gaube          Hotel: Veranda Paul et Virginie Hotel & Spa
City: Butaritari           Hotel: Isles Sunset Lodge
City: Wuzhou               Hotel: Wuzhou Hostel
City: Vaini                Hotel: Keleti Beach Resort
City: Matara               Hotel: Amaloh Boutique Resort
City: Avarua               Hotel: Paradise Inn
City: Rikitea              Hotel: Pension Maro'i
City: Hithadhoo            Hotel: Scoop Guest House
City: Lishu                Hotel: Shengyuan Hotel Fast Food
City: Makakilo City        Hotel: Marriott's Ko Olina Beach Club
City: Gumdag               Hotel: *** NO HOTEL FOUND *** ... Dropping row.
City: Ahuimanu             Hotel: Paradise Bay Resort
City: Djibo                Hotel: Centre Des Ressources DPENA
City: Manicore             Hotel: Pousada da wanda
City: Hua Hin              Hotel: InterContinental Hua Hin Resort
City: Abilene              Hotel: La Quinta Inn by Wyndham Abilene
City: Atuona   

City: Oistins              Hotel: Butterfly Beach Hotel
City: Nhulunbuy            Hotel: East Arnhem Real Estate
City: Campbell River       Hotel: Ramada by Wyndham Campbell River
City: Kismayo              Hotel: Agan-Hotel
City: Hobe Sound           Hotel: Floridays RV Park
City: Nouna                Hotel: Hôtel Chez Rose-Marie
City: Am Timan             Hotel: *** NO HOTEL FOUND *** ... Dropping row.
City: Port-Gentil          Hotel: Fengshui Residence
City: Thinadhoo            Hotel: *** NO HOTEL FOUND *** ... Dropping row.
City: Mareeba              Hotel: Trinity Plains Tourist Park
City: Cepu                 Hotel: OYO 673 Hotel Cepu Indah 1
City: Sulangan             Hotel: *** NO HOTEL FOUND *** ... Dropping row.
City: Naze                 Hotel: 奄美ポートタワーホテル
City: Biloela              Hotel: Silo Motor Inn
City: Takoradi             Hotel: Raybow International Hotel
City: Tezu                 Hotel: Circuit House
City: Morondava            Hotel: La Case Bambou
City: Manzan

In [13]:
# Reset index to renumber the rows sequencially since
# Cities where no nearby hotel was found were deleted
# from the data frame
preferred_cities_w_hotel_reset_df = preferred_cities_w_hotel_df.reset_index(drop=True)
preferred_cities_w_hotel_reset_df.head(30)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,80.60,scattered clouds,19.73,-155.09,Hilo Hawaiian Hotel
1,Grand Gaube,MU,70.00,scattered clouds,-20.01,57.66,Veranda Paul et Virginie Hotel & Spa
2,Butaritari,KI,83.68,scattered clouds,3.07,172.79,Isles Sunset Lodge
3,Wuzhou,CN,85.87,moderate rain,23.48,111.32,Wuzhou Hostel
4,Vaini,TO,80.60,thunderstorm,-21.20,-175.20,Keleti Beach Resort
5,Matara,LK,80.29,overcast clouds,5.95,80.54,Amaloh Boutique Resort
6,Avarua,CK,82.40,clear sky,-21.21,-159.78,Paradise Inn
7,Rikitea,PF,75.92,clear sky,-23.12,-134.97,Pension Maro'i
8,Hithadhoo,MV,84.63,light rain,-0.60,73.08,Scoop Guest House
9,Lishu,CN,70.02,clear sky,43.30,124.33,Shengyuan Hotel Fast Food


In [14]:
# Save the perferred cities with nearby hotels information to csv file
preferred_cities_w_hotel_df.to_csv("weather_data/WeatherPy_vacation.csv")

In [15]:
###########################################################
# Plot Perferred Cities With Nearby Hotels on Google Maps #
###########################################################

In [17]:
# Popup markers template for destination cities with hotel and weather information
destination_city_info_box_template = """
<dl>
    <dt>Hotel Name</dt>
        <dd>{Hotel Name}</dd>
    <dt>City</dt>
        <dd>{City}</dd>
    <dt>Country</dt>
        <dd>{Country}</dd>
    <dt>Current Weather</dt>
        <dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Create google map pop marker info boxes for destination cities.
destination_city_info_boxes = [destination_city_info_box_template.format(**row) for index, row in preferred_cities_w_hotel_df.iterrows()]

# Get destination cities locations
destination_city_locations = preferred_cities_w_hotel_df[["Lat", "Lng"]]

# # Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

# Add a pop-up marker for each city.
marker_layer = gmaps.marker_layer(destination_city_locations, info_box_content=destination_city_info_boxes)

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
fig

Figure(layout=FigureLayout(height='420px'))